In [1]:
!bash -c 'source activate tl-detect && conda install -qy -c conda-forge keras'

Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
  current version: 4.4.10
  latest version: 4.5.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda/envs/tl-detect

  added / updated specs: 
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    h5py-2.7.1                 |           py35_2         3.9 MB  conda-forge
    keras-2.1.5                |           py35_0         498 KB  conda-forge
    yaml-0.1.7                 |                0         302 KB  conda-forge
    pygpu-0.7.5                |           py35_0         1.4 MB  conda-forge
    mako-1.0.7                 |           py35_0         115 KB  conda-forge
    scipy-1.0.1                |py35_blas_openblas_200        39.4 MB  conda-forge
    markupsafe-1.0             |           py35_0          32

In [29]:
!cat ~/.keras/keras.json

{
    "backend": "tensorflow",
    "image_data_format": "channels_last",
    "epsilon": 1e-07,
    "floatx": "float32"
}

In [24]:
!cp -R ../imgs/training/bosch/Green ../imgs/training/ && cp -R ../imgs/training/bosch/Red ../imgs/training/ && cp -R ../imgs/training/bosch/Yellow ../imgs/training/  && mv ../imgs/training/bosch ../imgs/bosch_tl 

cp: cannot stat '../imgs/training/bosch/Green': No such file or directory


In [ ]:
! cd ../imgs/training && find . -size 0 -exec rm {} \;

In [12]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import keras

# dimensions of our images.
img_width, img_height = 75, 40


In [3]:
UNKNOWN = 4
GREEN = 2
YELLOW = 1
RED = 0

In [7]:
import cv2
import os
import numpy as np

In [10]:
def read_img(img_path, dim):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError('Couldn\'t read img: ' + img_path)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return cv2.resize(hsv, dim, interpolation=cv2.INTER_CUBIC)

def read_color(path, color):
    x = np.stack([read_img(path + f, (img_height, img_width)) for f in os.listdir(path)], axis=0)
    y = np.full(x.shape[0], color)
    return (x, y)

train_data_dir = '../imgs/training/'
x_train = []
y_train = []
for directory, color in zip(['Red/', 'Yellow/', 'Green/'], [RED, YELLOW, GREEN]):
    x, y = read_color(os.path.join(train_data_dir, directory), color)
    x_train.append(x)
    y_train.append(y)

x_train = np.concatenate(x_train)
y_train = np.concatenate(y_train)

validation_data_dir = '../imgs/validation/'
x_validation = []
y_validation = []
for directory, color in zip(['Red/', 'Yellow/', 'Green/'], [RED, YELLOW, GREEN]):
    x, y = read_color(os.path.join(validation_data_dir, directory), color)
    x_validation.append(x)
    y_validation.append(y)

x_validation = np.concatenate(x_validation)
y_validation = np.concatenate(y_validation)




In [13]:
x_train = x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_train /= 255
x_validation /= 255
NUM_CLASSES = 3

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_validation = keras.utils.to_categorical(y_validation, NUM_CLASSES)


In [119]:
x_train.shape

(7246, 75, 40, 3)

In [17]:
batch_size = 1000
epochs = 25

In [18]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_width, img_height)
    x_validation = x_validation.reshape(x_validation.shape[0], 3, img_width, img_height)
    input_shape = (3, img_width, img_heigth)
else:
    x_train = x_train.reshape(x_train.shape[0], img_width, img_height, 3)
    x_validation = x_validation.reshape(x_validation.shape[0], img_width, img_height, 3)
    input_shape = (img_width, img_height, 3)
    
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))
score = model.evaluate(x_validation, y_validation, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save_weights('first_try.h5')

Train on 7246 samples, validate on 893 samples
Epoch 1/25
7246/7246 [==============================] - 13s 2ms/step - loss: 0.9693 - acc: 0.5740 - val_loss: 1.1256 - val_acc: 0.3830
Epoch 2/25
7246/7246 [==============================] - 5s 659us/step - loss: 0.8316 - acc: 0.6105 - val_loss: 1.0584 - val_acc: 0.3830
Epoch 3/25
7246/7246 [==============================] - 4s 487us/step - loss: 0.8294 - acc: 0.6104 - val_loss: 1.2447 - val_acc: 0.3830
Epoch 4/25
7246/7246 [==============================] - 4s 486us/step - loss: 0.8213 - acc: 0.6116 - val_loss: 1.2048 - val_acc: 0.3830
Epoch 5/25
7246/7246 [==============================] - 4s 486us/step - loss: 0.8245 - acc: 0.6115 - val_loss: 1.2796 - val_acc: 0.3830
Epoch 6/25
7246/7246 [==============================] - 4s 487us/step - loss: 0.8260 - acc: 0.6116 - val_loss: 1.1093 - val_acc: 0.3830
Epoch 7/25
7246/7246 [==============================] - 4s 487us/step - loss: 0.8227 - acc: 0.6119 - val_loss: 1.1753 - val_acc: 0.3830
Ep